### Vanilla CTR Zeroshot

In [1]:
import pickle
import json
import pandas as pd
import numpy as np
import os
import time
from dotenv import load_dotenv
import re

In [2]:
inference_path = 'zeroshot_inference_test.json'
with open(inference_path, 'r') as f:
    inference_dict = json.load(f)
print(len(inference_dict))

620


In [3]:
ratings_path = '../beauty_data/processed_data/ratings.csv'
ratings_df = pd.read_csv(ratings_path)
ratings_df.head()

,item,user,rating,timestamp
0,0,0,4.0,1155081600
1,1,0,5.0,1268697600
2,2,0,5.0,1404604800
3,3,0,5.0,1404604800
4,4,0,5.0,1404604800


In [10]:
pattern = r"Prediction:(?:\s+)?(Yes|No)\b"
invalid_users = []
cnt = cntInvalid = cntY = cntN = 0
user_inference_dict = dict()
for user, inference in inference_dict.items():
    cnt += 1
    # print(user, inference)
    # Use re.findall to find all matches in the string
    matches = re.findall(pattern, inference, re.IGNORECASE)
    if len(matches) == 0:
        cntInvalid += 1
        invalid_users.append(user)
        user_inference_dict[user] = 1
    for match in matches:
        if match == 'Yes':
            cntY += 1
            user_inference_dict[user] = 1
            break
        elif match == 'No':
            cntN += 1
            user_inference_dict[user] = 0
            break
    # print('*'*100)
    # if cnt == 10:
    #     break
print(f"Yes: {cntY} -- No: {cntN} -- Invalid: {cntInvalid}")
print("invalid_users:", invalid_users)
print(len(user_inference_dict))

Yes: 113 -- No: 3 -- Invalid: 504
invalid_users: ['0', '3', '4', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '19', '20', '22', '23', '24', '25', '27', '28', '29', '31', '32', '35', '36', '37', '38', '39', '40', '41', '44', '46', '47', '48', '49', '51', '52', '53', '54', '55', '56', '57', '58', '61', '62', '63', '64', '65', '66', '67', '68', '69', '71', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '85', '86', '87', '88', '89', '90', '91', '93', '94', '95', '97', '98', '99', '100', '101', '103', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '120', '121', '122', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '136', '137', '138', '141', '142', '143', '145', '146', '148', '149', '150', '152', '153', '155', '156', '157', '159', '160', '164', '165', '166', '168', '172', '173', '174', '175', '177', '178', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189',

In [11]:
user_test_movie_rating_df = ratings_df.groupby('user').nth(-1)
user_test_movie_rating_df.head()

,item,user,rating,timestamp
7,6,0,5.0,1461801600
14,12,1,5.0,1504656000
21,12,2,4.0,1506643200
27,20,3,5.0,1493337600
33,21,4,5.0,1524614400


In [12]:
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score
y_true = []
y_pred = []

fp_users = []
fn_users = []

for user, inference in user_inference_dict.items():
    original_rating = user_test_movie_rating_df[user_test_movie_rating_df['user'] == int(user)]['rating'].values[0]
    # print(original_rating)
    if original_rating > 3:
        y_true.append(1)
        if inference == 0:
            fn_users.append(user)
    else:
        y_true.append(0)
        if inference == 1:
            fp_users.append(user)
    y_pred.append(inference)

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

auc_roc = roc_auc_score(y_true, y_pred)
print("AUC:", auc_roc)

precision = precision_score(y_true, y_pred)
print("Precision:", precision)

recall = recall_score(y_true, y_pred)
print("Recall:", recall)

accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

Confusion Matrix:
[[  1  43]
 [  2 574]]
AUC: 0.5096275252525253
Precision: 0.9303079416531604
Recall: 0.9965277777777778
Accuracy: 0.9274193548387096


In [13]:
user_valid_movie_rating_df = ratings_df.groupby('user').nth(-2)
user_valid_movie_rating_df.head()

,item,user,rating,timestamp
6,5,0,5.0,1404604800
13,11,1,5.0,1504656000
20,11,2,4.0,1506643200
26,19,3,5.0,1493337600
32,19,4,5.0,1524614400


In [14]:
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score
y_true = []
y_pred = []

fp_users = []
fn_users = []

for user, inference in user_inference_dict.items():
    original_rating = user_valid_movie_rating_df[user_valid_movie_rating_df['user'] == int(user)]['rating'].values[0]
    # print(original_rating)
    if original_rating > 3:
        y_true.append(1)
        if inference == 0:
            fn_users.append(user)
    else:
        y_true.append(0)
        if inference == 1:
            fp_users.append(user)
    y_pred.append(inference)

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

auc_roc = roc_auc_score(y_true, y_pred)
print("AUC:", auc_roc)

precision = precision_score(y_true, y_pred)
print("Precision:", precision)

recall = recall_score(y_true, y_pred)
print("Recall:", recall)

accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

Confusion Matrix:
[[ 31  15]
 [321 257]]
AUC: 0.5592748608394764
Precision: 0.9448529411764706
Recall: 0.444636678200692
Accuracy: 0.46153846153846156
